This file contains some functions to check that ICD, OPERTN, MAINSPEF and PROCODES codes match with the reference tables. Functions are tested on APC tables (positive and negative).

# Connection

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import numpy as np

%matplotlib inline

In [2]:
def connect():
    return pyodbc.connect(
        'DRIVER={/usr/lib/x86_64-linux-gnu/odbc/libtdsodbc.so}; SERVER=192.168.5.78;'
        'DATABASE=IMS-NHSDigital;UID=sa;PWD=2HsxpmkDLSlHlT;port=1433;'
        'TDS_Version=8.0;')
engine = sqlalchemy.create_engine('mssql+pyodbc://', creator=connect)
conn = engine.connect()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/sqlalchemy/connectors/pyodbc.py:92: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  "No driver name specified; "


# Check on PROCODES

Extract the unique set of hospitals (PROCODE) codes and provide patient count for each. Join this with the reference table for the PROCODE codes and flag those that do not match a code in the reference table.

In [364]:
#extract codes
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

#table_list=['dbo.APC_Linkage']

label_ID="PROCODE"

unique_procodes=set()
for tablename in table_list:
    query= 'SELECT ' + label_ID  +' from ' + tablename
    print(query)
    procodes=pd.read_sql(query, conn)[label_ID]
    unique_procodes.update(procodes)
    print(len(unique_procodes))

#print(unique_procodes)
#I suggest to count for those not matching if we just care about them

SELECT PROCODE from dbo.NIC58999_APC_Linkage_201299
190
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201399
244
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201499
275
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201599
296
SELECT PROCODE from dbo.NIC58999_APC_Linkage_201613
310


In [365]:
#if I use the procode col the majority of codes are 5 digits,
#sometime endings with 00/01, sometimes with other letters or number
list(unique_procodes)[0:12]

['RJD00',
 'RM300',
 'RWJ00',
 'RXE',
 'RXM00',
 'RTG',
 'NVC20',
 'RT200',
 'RQM00',
 'RTR00',
 'NPG06',
 'RW401']

In [366]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimProvider.csv'
ref_label_procode='ProviderSiteCode'
ref_procode=pd.read_csv(ref_table, delimiter="|")[ref_label_procode].unique()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [367]:
#the majority of reference codes are 3digit codes, seems sensible to use the procode3 from HES tables
ref_procode[:30]

array(['----------------', 'Missing', 'Unknown', '5A3', '5A301', '5A302',
       '5A303', '5A305', '5A306', '5A307', '5A308', '5A309', '5A310',
       '5A312', '5A313', '5A314', '5A315', '5A316', '5A317', '5A318',
       '5A319', '5A320', '5A321', '5A322', '5A323', '5A324', '5A325',
       '5A326', '5A327', '5A328'], dtype=object)

In [260]:
len(set(ref_procode))

452

In [261]:
#some codes are not matching
unique_procodes.issubset(set(ref_procode))

False

In [368]:
#about 25% of coding is not matching with the full dataset
#if I use procode is about 90% ...
NM_procodes=[x for x in unique_procodes if x not in ref_procode]
print(len(NM_procodes)/len(unique_procodes))
print(NM_procodes)

0.34838709677419355
['RJD00', 'RM300', 'RWJ00', 'RXM00', 'RT200', 'RQM00', 'RTR00', 'RXT00', 'RAN00', 'RR100', 'RA900', 'RBK00', 'RVL00', 'R1D00', 'RTX00', 'RX100', 'RYR00', 'RD100', 'RTP00', 'RRE00', 'R1K00', 'RTE00', 'RTH00', 'RBA00', 'RA300', 'RM200', 'RYY00', 'RXE00', 'RKE00', 'RN500', 'RQQ00', 'RKB00', 'RXF00', 'RP700', 'RCX00', 'RWA00', 'RXN00', 'RXP00', 'RJL00', 'RMP00', 'RM100', 'RR800', 'RHM00', 'RFR00', 'RN300', 'RJR00', 'RLU00', 'RK900', 'RWP00', 'RL100', 'RAL00', 'RVW00', 'RGP00', 'RGQ00', 'RLT00', 'R1H00', 'RNS00', 'RXH00', 'RH800', 'RRJ00', 'RAX00', 'RNQ00', 'RXL00', 'TAH00', 'RVR00', 'RTF00', 'TAJ00', 'RHQ00', 'RQ600', 'RT300', 'RGR00', 'RBV00', 'RNL00', 'RMC00', 'RJC00', 'R1E00', 'RP600', 'RN700', 'RYJ00', 'RGN00', 'RP400', 'RBL00', 'RBT00', 'RA400', 'RXQ00', 'RF400', 'RW600', 'RAE00', 'RBQ00', 'REM00', 'RV800', 'R1F00', 'RTD00', 'RXC00', 'RR700', 'RWW00', 'RWG00', 'RWE00', 'RL400', 'RWY00', 'RW300', 'RW500', 'RGD00', 'RQ300', 'RRF00', 'RYV00', 'REF00', 'RPY00']


In [84]:
ref_code_8=[x for x in ref_procode if x.startswith('8')]

In [85]:
ref_code_8

['8A101',
 '8E339',
 '8F689',
 '8HP32',
 '8HP46',
 '8HT55',
 '8HT56',
 '8J124',
 '8J244',
 '8J248',
 '8J285',
 '8G301']

In [86]:
#we might assume that 8E3 stands for 8A101 and 8E3 stands for 8E339
#for what we use these codes after? useful to join with Provider Caption (contains the name of the provider)?

In [151]:
#count occurencies for not matching codes
NM_df=pd.DataFrame({label_ID: [],'counts': []})
for code in NM_procodes:
    for tablename in table_list:
        query= 'SELECT ' + label_ID + ', COUNT(' + label_ID + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_ID + ' IS NOT NULL AND ' + label_ID +' = \'' + code + '\' GROUP BY ' + label_ID + ';'  
        #print(query)
        NM_df=NM_df.append(pd.read_sql(query, conn), ignore_index=True)

In [156]:
#grouping the counts from different tables
NM_df.groupby(label_ID)['counts'].sum()

PROCODE3
8A1    10.0
8E3     3.0
Name: counts, dtype: float64

In [ ]:
#how often these codes appear?
for tablename in table_list:
        query= 'SELECT COUNT(' + label_ID + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_ID + ' IS NOT NULL AND ' + label_ID +' = \'' + code + '\' GROUP BY ' + label_ID + ';'  
        
print(len(NM_procodes)/len(unique_procodes))

# Check TRETSPEF

Extract the unique set of main specialties (TRETSPEF) codes and provide count for each. Join this with the reference table for the MAINSPEF codes and flag those that do not match a code in the reference table.  **ACHTUNG** we changed from MAINSPEF to TRETSPEF, that is the actual code the consultant treated the patient for

In [266]:
#extract codes
#MAINSPEF is always a 3digit code or &(meaning not known)
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

label_spec="TRETSPEF"

unique_spec=set()
for tablename in table_list:
    query= 'SELECT ' + label_spec  +' from ' + tablename
    spec=pd.read_sql(query, conn)[label_spec]
    unique_spec.update(spec)
    #print(len(unique_mainspef))

In [267]:
#reading reference codes
#Missing and unknown labelled differently from &
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimSpecialties.csv'
ref_label_spec='SpecialtyCode'
ref_spec=pd.read_csv(ref_table, delimiter="|")[ref_label_spec].unique()

In [268]:
ref_spec

array(['-------------', 'Miss', 'Unk', '000', '100', '101', '102', '103',
       '104', '105', '106', '107', '108', '110', '120', '130', '140',
       '141', '142', '143', '144', '145', '146', '147', '148', '149',
       '150', '160', '161', '170', '171', '172', '173', '174', '180',
       '190', '191', '192', '199', '211', '212', '213', '214', '215',
       '216', '217', '218', '219', '220', '221', '222', '223', '241',
       '242', '251', '252', '253', '254', '255', '256', '257', '258',
       '259', '260', '261', '262', '263', '264', '280', '290', '291',
       '300', '301', '302', '303', '304', '305', '306', '307', '308',
       '309', '310', '311', '312', '313', '314', '315', '316', '317',
       '318', '319', '320', '321', '322', '323', '324', '325', '326',
       '327', '328', '329', '330', '331', '340', '341', '342', '343',
       '344', '345', '346', '350', '351', '352', '360', '361', '370',
       '371', '400', '401', '410', '420', '421', '422', '424', '430',
       '450', '4

In [269]:
unique_spec.issubset(set(ref_spec))

False

In [270]:
#& is unknow
NM_spec=[x for x in unique_spec if x not in ref_spec]
print(len(NM_spec)/len(unique_spec))
print(NM_spec)

0.011494252873563218
['&']


In [271]:
#This was for MAINSPEF
#No idea how to match 451...maybe 450 by proximity?
#ref_spec_4=[x for x in ref_spec if x.startswith('4')]
#ref_spec_4
#no reference codes containing the substring
#ref_spec_451=[x for x in ref_spec if '451' in x]
#ref_spec_451

In [290]:
#count occurencies for not matching codes
NM_df=pd.DataFrame({label_spec: [],'counts': []})
for code in NM_spec:
    for tablename in table_list:
        query= 'SELECT ' + label_spec + ', COUNT(' + label_spec + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_spec + ' IS NOT NULL AND ' + label_spec +' = \'' + code + '\' GROUP BY ' + label_spec + ';'  
        #print(query)
        NM_df=NM_df.append(pd.read_sql(query, conn), ignore_index=True)

In [291]:
#grouping the counts from different tables
NM_df.groupby(label_spec)['counts'].sum()

TRETSPEF
&    17
Name: counts, dtype: int64

In [314]:
Nevents=[]
for tablename in table_list:
    query= 'SELECT COUNT(*) from ' + tablename  + ';' 
    #print(query)
    #print(pd.read_sql(query, conn))
    Nevents.append(pd.read_sql(query, conn).iloc[:,0])
Nevents=sum(Nevents)
print(Nevents)

0    67597
Name: , dtype: int64


In [313]:
#Final tables are merged, not worty to code to compute the aggregration from different tables
17/Nevents

0    0.000251
Name: , dtype: float64

# Check OPERTN

Extract the unique set of procedure codes (OPERTN) codes and provide count for each. Join this with the reference table for the procedure codes and flag those that do not match a code in the reference table.  

In [330]:
#extract codes
#OPERTN contains a 4digit code, a letter followed by 3 digits (it might be only 2 digits according to the book)
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

label_proc_list=["OPERTN_" + str(i).zfill(2) for i in range(1,25)]

unique_proc=set()
for tablename in table_list:
    for label_proc in label_proc_list:
        query= 'SELECT ' + label_proc  +' from ' + tablename
        proc=pd.read_sql(query, conn)[label_proc]
        unique_proc.update(proc)
        #print(len(unique_mainspef))

In [322]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimProcedure.csv'
#4th digit is a variation of the main procedure
ref_label_proc='ProcedureLevel4Code'
#unique() is creating some issues, the same thing can be achieved with set
ref_proc=set(pd.read_csv(ref_table, delimiter="|", encoding = "ISO-8859-1")[ref_label_proc])

In [331]:
unique_proc.issubset(set(ref_proc))

False

In [350]:
NM_proc=[x for x in unique_proc if x not in ref_proc]
print(len(NM_proc)/len(unique_proc))
print(NM_proc)

0.0023594180102241447
['', 'O302', 'O301', 'O281', '-', '&']


In [358]:
label_proc_list

['OPERTN_01',
 'OPERTN_02',
 'OPERTN_03',
 'OPERTN_04',
 'OPERTN_05',
 'OPERTN_06',
 'OPERTN_07',
 'OPERTN_08',
 'OPERTN_09',
 'OPERTN_10',
 'OPERTN_11',
 'OPERTN_12',
 'OPERTN_13',
 'OPERTN_14',
 'OPERTN_15',
 'OPERTN_16',
 'OPERTN_17',
 'OPERTN_18',
 'OPERTN_19',
 'OPERTN_20',
 'OPERTN_21',
 'OPERTN_22',
 'OPERTN_23',
 'OPERTN_24']

In [361]:
#count occurencies for not matching codes
NM_df=pd.DataFrame({'OPERTN': [],'counts': []})
for code in NM_proc:
    lab_counts=0
    print(code)
    for tablename in table_list:
        for  label_proc in label_proc_list:
            query= 'SELECT  COUNT(' + label_proc + ') AS counts' + ' from ' + tablename + \
            ' WHERE '+ label_proc + ' IS NOT NULL AND ' + label_proc +' = \'' + code + '\' GROUP BY ' + label_proc + ';'  
            df=pd.read_sql(query, conn)
            if (df.empty==False):
                count_col= df['counts'][0]
                lab_counts=lab_counts+count_col
                #print(label_proc,lab_counts)
            count_col=0
    NM_df=NM_df.append(pd.DataFrame({'OPERTN': [code],'counts': [lab_counts]}), ignore_index=True)


O302
O301
O281
-
&


In [362]:
NM_df

,OPERTN,counts
0,,1491069.0
1,O302,76.0
2,O301,18.0
3,O281,2.0
4,-,30504.0
5,&,1.0


In [335]:
NM_df

,OPERTN_01,OPERTN_02,OPERTN_03,OPERTN_04,OPERTN_05,OPERTN_06,OPERTN_07,OPERTN_08,OPERTN_09,OPERTN_10,...,OPERTN_16,OPERTN_17,OPERTN_18,OPERTN_19,OPERTN_20,OPERTN_21,OPERTN_22,OPERTN_23,OPERTN_24,counts
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8988
2,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10665
3,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11663
4,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12223
5,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12516
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12714
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12915
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13023
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13085


In [ ]:
#grouping the counts from different tables
NM_df.groupby('Proc')['counts'].sum()

# Check ICD

In [222]:
#extract codes
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

#TO BE GENERALIZED: we have to append the codes in ALL the opertn cols
label_diag="DIAG_01"

unique_diag=set()
for tablename in table_list:
    query= 'SELECT ' + label_diag  +' from ' + tablename
    diag=pd.read_sql(query, conn)[label_diag]
    unique_diag.update(diag)

In [ ]:
list(unique_diag)[:100]

In [225]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimDiagnosis.csv'
#4th digit is a variation of the main procedure
ref_label_diag='DiagnosisLevel4Code'
ref_diag=pd.read_csv(ref_table, delimiter="|",encoding = "ISO-8859-1")[ref_label_diag].unique()

In [226]:
ref_diag

array(['-------------------', 'Missing', 'Unknown', ..., 'U848', 'U849',
       'U85X'], dtype=object)